In [1]:
from DeconTools.core.Pupil import PupilFunction
from DeconTools.core.ImagingModels import CarringtonModel
from DeconTools.viz import SimpleOrthoViewer
import ndv
import numpy as np
import matplotlib.pyplot as plt
import tifffile
import torch
from imaris_file import ImarisReader
%load_ext autoreload
%autoreload 2
# %matplotlib qt

2025-02-20 06:43:11,565 - DeconTools - INFO - Logging level set to INFO


In [16]:
with ImarisReader('/Users/delnatan/BurgessLab/live_imaging/yeastTL_1_F1_crop.ims') as ims:
    frame27 = ims.get_data(time_point=27)[:,200:200+100, 150:150+100]
    print(frame27.shape)
v = SimpleOrthoViewer(frame27)

(21, 100, 100)


In [4]:
opars = {
    "NA": 0.95,
    "ni": 1.0,
    "ns": 1.33,
    "wavelength": 0.529,
    "dz": 0.381,
    "dxy": 0.1565,
    "source_depth": 3.0
}
m = CarringtonModel(frame27.shape, (2.0, 3.0, 3.0), opars, device_str="cpu")

In [5]:
MPS = torch.device("cpu")
d = torch.from_numpy(
    np.maximum(frame27.astype(np.float32) - 100.0, 1e-5)
).to(MPS)

In [6]:
xest = torch.ones(m.padded_object_shape, dtype=torch.float32, device=MPS) * d.mean()
hnorm = m.adjoint(torch.ones(m.data_shape, dtype=torch.float32, device=MPS))    

In [8]:
for k in range(50):
    print(f"iter = {k}")
    model = m.forward(xest)
    ratio = d / model
    update = m.adjoint(ratio)
    xest *= update
    xest = torch.where(hnorm > 1e-4, xest / hnorm, xest)

iter = 0
iter = 1
iter = 2
iter = 3
iter = 4
iter = 5
iter = 6
iter = 7
iter = 8
iter = 9
iter = 10
iter = 11
iter = 12
iter = 13
iter = 14
iter = 15
iter = 16
iter = 17
iter = 18
iter = 19
iter = 20
iter = 21
iter = 22
iter = 23
iter = 24
iter = 25
iter = 26
iter = 27
iter = 28
iter = 29
iter = 30
iter = 31
iter = 32
iter = 33
iter = 34
iter = 35
iter = 36
iter = 37
iter = 38
iter = 39
iter = 40
iter = 41
iter = 42
iter = 43
iter = 44
iter = 45
iter = 46
iter = 47
iter = 48
iter = 49


In [9]:
xest_unpad = m.output_padder.adjoint(xest)

In [11]:
%matplotlib qt

In [12]:
p99 = np.percentile(xest_unpad.cpu().numpy(), 99.99)
v = SimpleOrthoViewer(xest_unpad.cpu().numpy(), 
                      norm=plt.Normalize(vmin=0, vmax=p99))

2025-02-20 06:46:07.391 python[15897:1286405] +[IMKClient subclass]: chose IMKClient_Modern
2025-02-20 06:46:07.391 python[15897:1286405] +[IMKInputSession subclass]: chose IMKInputSession_Modern


In [12]:
xest_unpad.max()

tensor(nan, device='mps:0')

In [13]:
v = SimpleOrthoViewer(d.cpu().numpy())